<a class="reference external" 
    href="https://jupyter.designsafe-ci.org/hub/user-redirect/lab/tree/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks/tapis_appsDev_CustomApp_opsmps3copy.ipynb" 
    target="_blank"
    >
<img alt="Try on DesignSafe" src="https://raw.githubusercontent.com/DesignSafe-Training/pinn/main/DesignSafe-Badge.svg" /></a>

# Agnostic App  📒
***Create your own Tapis App***

by Silvia Mazzoni, DesignSafe, 2025

This is a step-by-step guide to help you write your own **Tapis v3 App** — from defining the app, to registering it, and running it. 

This is a practical walkthrough for defining and deploying an **HPC** app using the Tapis v3 API.

In this notebook we will modify the opensees-mp-s3 for OpenSeesMP or OpenSeesSP, etc.

## Workflow

| Step                        | Description                                                         |
| --------------------------- | ------------------------------------------------------------------- |
| 1. Create *app.json*        | Describes the app, its inputs, execution system, and wrapper script |
| 2. Create *tapisjob_app.sh* | Runs your analysis (e.g., ibrun OpenSees main.tcl)                  |
| 2a. Zip *tapisjob_app.sh* | It needs a zip file!                  |
| 3. Create *profile.json*    | (Optional) Loads modules/environment                                |
| 4. Upload Files             | To the deployment path in your storage system                       |
| 5. Register App             | With Tapis via CLI or Python                                        |
| 6. Submit Job               | Define `job.json` and submit                                        |

In [1]:
pwd

'/home/jupyter/MyData/_ToCommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks'

In [2]:
import json

In [3]:
# Local Utilities Library
# you can remove the logic associated with the local path
import sys,os
relativePath = '../OpsUtils'
if os.path.exists(relativePath):
    print("Using local utilities library")
    PathOpsUtils = os.path.expanduser(relativePath)
else:
    print('using communitydata')
    PathOpsUtils = os.path.expanduser('~/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/OpsUtils')
if not PathOpsUtils in sys.path: sys.path.append(PathOpsUtils)
from OpsUtils import OpsUtils

Using local utilities library


---
## Connect to Tapis

In [4]:
t=OpsUtils.connect_tapis()

 -- Checking Tapis token --
 Token loaded from file. Token is still valid!
 Token expires at: 2025-08-20T01:34:27+00:00
 Token expires in: 2:37:21.595649
-- LOG IN SUCCESSFUL! --


---
### Get username

In [5]:
OpsUtils.show_text_file_in_accordion(PathOpsUtils, ['get_tapis_username.py'])

In [6]:
username = OpsUtils.get_tapis_username(t)
print('username',username)

username silvia


In [7]:
## Define actions!

In [8]:
# Make these True once you are done validating
do_makeApp = True
do_submitJob = True

---
## Configure App

In [9]:
app_id = 'agnostic-app-test'
app_description = 'Run any Executable'

### AutoIncrement app_version

In [10]:
if do_makeApp:
    latest_app_version = OpsUtils.get_latest_app_version(t,app_id)
    print('now latest_app_version',latest_app_version)
    app_version = OpsUtils.bump_app_version(latest_app_version,'patch')
    print('now app_version',app_version)

now latest_app_version 0.0.18
now app_version 0.0.19


### Where to store the files

In [11]:
if do_makeApp:
    app_system_id="designsafe.storage.default"
    app_path = f"{username}/apps/{app_id}/{app_version}"
    container_filename = f'{app_id}.zip'

### Define folder structure: app_id/version

In [12]:
if do_makeApp:
    app_folderName = f'~/MyData/myAuthoredTapisApps/{app_id}/{app_version}'; # your choice

In [13]:
if do_makeApp:
    app_folder = os.path.abspath(os.path.expanduser(app_folderName))
    os.makedirs(app_folder, exist_ok=True)
    print(f'app_folder: {app_folder}\n exists:',os.path.exists(app_folder))

app_folder: /home/jupyter/MyData/myAuthoredTapisApps/agnostic-app-test/0.0.19
 exists: True


---
## Create the App Files

A Tapis app needs the following **core files**:

* **Readme.MD** – App Description (OPTIONAL)<br>
This file is helpful in communicating content to the app user.  

* **app.json** – App Definition<br>
Defines the app’s metadata, inputs, parameters, and execution configuration.    

* **profile.json** – Environment Setup *(optional but common)*<br>
Loads modules or sets environment variables on the execution system. This file will load modules before your script runs. This is executed on the compute node.

* **tapisjob_app.sh** – Wrapper Script<br>
Wrapper script executed by the job; this is the command that launches your code (e.g., runs *OpenSeesMP*, Python, or a script)

In [14]:
# let's collect the list of files we are creating as we will need this list in upload
if do_makeApp:
    appFilesList = [];

#### **Readme.MD** – App Description
This file is helpful in communicating content to the app user.  

In [15]:
if do_makeApp:
    thisFilename = 'ReadMe.MD'
    thisText = """\
    # making an application-agnostic app
    """
    
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

    appFilesList.append(thisFilename)

#### **app.json** – App Definition
Defines the app’s metadata, inputs, parameters, and execution configuration.    

In [16]:
if do_makeApp:
    thisFilename = 'app.json'
    thisText = """\
    {
      "id": "__app_id__",
      "version": "__app_version__",
      "description": "__app_description__",
      "owner": "${apiUserId}",
      "enabled": true,
      "runtime": "ZIP",
      "runtimeVersion": null,
      "runtimeOptions": null,
      "containerImage": "__container_filename__",
      "jobType": "BATCH",
      "maxJobs": -1,
      "maxJobsPerUser": -1,
      "strictFileInputs": true,
      "jobAttributes": {
        "execSystemConstraints": null,
        "execSystemId": "stampede3",
        "execSystemExecDir": "${JobWorkingDir}",
        "execSystemInputDir": "${JobWorkingDir}",
        "execSystemOutputDir": "${JobWorkingDir}",
        "execSystemLogicalQueue": "skx",
        "archiveSystemId": "stampede3",
        "archiveSystemDir": "HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}",
        "archiveOnAppError": true,
        "isMpi": false,
        "mpiCmd": null,
        "parameterSet": {
          "appArgs": [
            {
              "name": "Main Program",
              "description": "Choose the binary Executable to use.",
              "arg": "OpenSeesMP",
              "inputMode": "REQUIRED",
                "notes": {
                    "enum_values": [
                        {
                            "OpenSeesMP": "OpenSeesMP"
                        },
                        {
                            "OpenSeesSP": "OpenSeesSP"
                        },
                        {
                            "python": "python"
                        }
                    ]
                }
            },
            {
              "name": "Main Script",
              "description": "The filename only of the input script to submit to executable. This file should reside in the Input Directory specified.",
              "arg": null,
              "inputMode": "REQUIRED",
              "notes": {
                "inputType": "fileInput"
              }
            },
            {
              "name": "CommandLine Arguments",
              "description": "Command-Line Arguments (Optional)",
              "arg": null,
              "inputMode": "INCLUDE_ON_DEMAND",
              "notes": {
              }
            }
            
          ],
          "containerArgs": [],
          "schedulerOptions": [
            {
              "name": "OpenSees TACC Scheduler Profile",
              "description": "Scheduler profile for the default version of OpenSees",
              "inputMode": "FIXED",
              "arg": "--tapis-profile OpenSees_default",
              "notes": {
                "isHidden": true
              }
            }
          ],
          "envVariables": [
                  { "key": "zipFileIn", "value": "", "inputMode": "INCLUDE_BY_DEFAULT" },
                  { "key": "zipFolderOut", "value": "False", "inputMode": "INCLUDE_BY_DEFAULT" },
                  { "key": "requirementFile", "value": "", "inputMode": "INCLUDE_BY_DEFAULT" }
             ],
          "archiveFilter": {
            "includes": [],
            "excludes": [],
            "includeLaunchFiles": true
          }
        },
        "fileInputs": [
          {
            "name": "Input Directory",
            "inputMode": "REQUIRED",
            "sourceUrl": null,
            "targetPath": "inputDirectory",
            "envKey": "inputDirectory",
            "description": "Input directory that includes the input script as well as any other required files. Example input is in tapis://designsafe.storage.community/app_examples/opensees/OpenSeesMP",
            "notes": {
              "selectionMode": "directory"
            }
          }
        ],
        "fileInputArrays": [],
        "nodeCount": 1,
        "coresPerNode": 48,
        "memoryMB": 192000,
        "maxMinutes": 120,
        "subscriptions": [],
        "tags": []
      },
      "tags": [
        "portalName: DesignSafe",
        "portalName: CEP"
      ],
      "notes": {
        "label": "__app_id__",
        "helpUrl": "",
        "hideNodeCountAndCoresPerNode": false,
        "isInteractive": false,
        "icon": "OpenSees",
        "category": "Simulation"
      }
    }
    """
    thisText = thisText.replace("__app_id__", app_id)
    thisText = thisText.replace("__app_version__", app_version)
    thisText = thisText.replace("__app_description__", app_description)
    thisText = thisText.replace("__container_filename__", f"tapis://{app_system_id}/{app_path}/{container_filename}")
    
    
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

    appFilesList.append(thisFilename)

#### **profile.json** – Environment Setup
This file will load modules before your script runs. This is executed on the compute node.

In [17]:
if do_makeApp:
    thisFilename = 'profile.json'
    thisText = """\
    {
        "name": "__app_id__",
        "description": "Modules to load",
        "moduleLoads": [ ],
        "hiddenOptions": [
            "MEM"
        ]
    }
    """    
    # thisText = """\
    # {
    #     "name": "AgnosticApp",
    #     "description": "Modules to load",
    #     "moduleLoads": [
    #         {
    #             "modulesToLoad": [
    #                 "hdf5/1.14.4",
    #                 "opensees"
    #             ],
    #             "moduleLoadCommand": "module load"
    #         }
    #     ],
    #     "hiddenOptions": [
    #         "MEM"
    #     ]
    # }
    # """
    thisText = thisText.replace("__app_id__", app_id)
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

    appFilesList.append(thisFilename)

#### **tapisjob_app.sh** – Wrapper Script
Wrapper script executed by the job; this is the command that launches your code (e.g., runs *OpenSeesMP*, Python, or a script)

In [18]:
if do_makeApp:
    import textwrap, time
    from zipfile import ZipFile, ZIP_DEFLATED, ZipInfo
    
    thisFilename_sh = "tapisjob_app.sh"
    thisFilename = container_filename  # existing var


    bash_script = textwrap.dedent("""\
        #!/bin/bash
        set -euo pipefail
        set -x

        # ---- required args ----
        BINARYNAME="${1:?missing binary name}"
        INPUTSCRIPT0="${2:?missing input script}"
        shift 2

        # ---- optional env with safe defaults ----
        inputDirectory="${inputDirectory:?inputDirectory not set}"
        zipFileIn="${zipFileIn:-}"
        zipFolderOut="${zipFolderOut:-}"
        echo "zipFileIn: ${zipFileIn}"
        echo "zipFolderOut: ${zipFolderOut}"

        INPUTSCRIPT="${INPUTSCRIPT0##*/}"
        echo "INPUTSCRIPT: $INPUTSCRIPT"
        echo "ARGS: $*"

        cd -- "$inputDirectory"

        # ---- expand input ZIP if provided ----
        if [[ -n "$zipFileIn" && -f "$zipFileIn" ]]; then
          echo "Unzipping $zipFileIn ..."
          unzip -o -q "$zipFileIn"
        fi

        # ---- modules for OpenSees family ----
        case "$BINARYNAME" in
          OpenSees|OpenSeesMP|OpenSeesSP)
            module load hdf5/1.14.4
            module load opensees
            module list || true
            echo "Loaded OpenSees modules"
            ;;
        esac

        # ---- modules for python ----
        if [[ -n "$requirementFile" && -f "$requirementFile" ]]; then
          echo "Install requirements file"
          python -m pip install -r "$requirementFile"
        fi

        # ---- choose launcher ----
        if [[ "$BINARYNAME" == "OpenSees" ]]; then
          LAUNCH=()        # direct run for sequential
        else
          LAUNCH=(ibrun)   # MPI/SP use ibrun on TACC
        fi

        echo "Running: ${LAUNCH[*]} $BINARYNAME $INPUTSCRIPT $*"
        if ! "${LAUNCH[@]}" "$BINARYNAME" "$INPUTSCRIPT" "$@"; then
          rc=$?
          echo "Program exited with error status: $rc" >&2
          exit "$rc"
        fi
        echo "done, now zippit!"
        # ---- optional re-pack an output folder ----
        if [[ "${zipFolderOut:-}" =~ ^([Tt][Rr][Uu][Ee]|1|[Yy][Ee]?[Ss]?)$ ]]; then
          cd ..
          echo "Zipping output folder "$inputDirectory" -> ${zipFolderOut}.zip"
          zip -r -q "inputDirectory.zip" "$inputDirectory"
          rm -rf "$inputDirectory"
        fi
    """)


    zip_path = os.path.join(app_folder, thisFilename)
    print('zip_path',zip_path)

    # write with executable permission inside the ZIP
    zi = ZipInfo(thisFilename_sh)
    zi.date_time = time.localtime(time.time())[:6]
    zi.compress_type = ZIP_DEFLATED
    zi.external_attr = 0o100755 << 16   # -rwxr-xr-x on a regular file
    
    with ZipFile(zip_path, "w", ZIP_DEFLATED) as z:
        z.writestr(zi, bash_script)
        
    appFilesList.append(thisFilename)

zip_path /home/jupyter/MyData/myAuthoredTapisApps/agnostic-app-test/0.0.19/agnostic-app-test.zip


#### File Check
Look at the files we have written and check for typos or formatting errors.

In [19]:
if do_makeApp:
    print(appFilesList)
    OpsUtils.show_text_file_in_accordion(app_folder, appFilesList)

['ReadMe.MD', 'app.json', 'profile.json', 'agnostic-app-test.zip']


---
## Validate App Locally

In [20]:
if do_makeApp:
    OpsUtils.show_text_file_in_accordion(PathOpsUtils, ['validate_app_folder.py'])

In [21]:
if do_makeApp:
    validation = OpsUtils.validate_app_folder(app_folder,appFilesList)
    if not validation:
        print('Validation Failed: stopping here!!!!')
        a = 3/0

🔍 Validating app folder: /home/jupyter/MyData/myAuthoredTapisApps/agnostic-app-test/0.0.19

✅ All required files are present.

📄 App ID: agnostic-app-test
📄 App Name: agnostic-app-test
📄 Version: 0.0.19
🔧 Parameters: []
📦 Inputs: []
📤 Outputs: []

App Keys: ['id', 'version', 'description', 'owner', 'enabled', 'runtime', 'runtimeVersion', 'runtimeOptions', 'containerImage', 'jobType', 'maxJobs', 'maxJobsPerUser', 'strictFileInputs', 'jobAttributes', 'tags', 'notes', 'name', 'moduleLoads', 'hiddenOptions']

✅ Basic validation complete. App folder looks good!


---
## Deploy the App

### Make the directory inside your 'MyData' on designsafe.storage.default.
The apps in this folder are the ones that area actually uploaded.

In [22]:
if do_makeApp:
    t.files.mkdir(systemId=app_system_id, path=app_path)
    print('app_path',app_path)

app_path silvia/apps/agnostic-app-test/0.0.19


### Upload files to your deployment system (e.g., DesignSafe default storage)
Using Tapipy (Python SDK) in a Jupyter Notebook

In [23]:
if do_makeApp:
    for fname in appFilesList:
        fpath = f'{app_folder}/{fname}'
        t.upload(source_file_path=fpath,
                 system_id=app_system_id,
                 dest_file_path=f'{app_path}/{fname}')

#### Verify upload

In [24]:
if do_makeApp:
    print('app_system_id:',app_system_id)
    print('app_path:',app_path)
    appfiles = t.files.listFiles(systemId=app_system_id, path=app_path)
    for thisF in appfiles:
        print(thisF)
        print('')

app_system_id: designsafe.storage.default
app_path: silvia/apps/agnostic-app-test/0.0.19

group: 819066
lastModified: 2025-08-19T22:57:07Z
mimeType: application/zip
name: agnostic-app-test.zip
nativePermissions: rw-rw----
owner: 843714
path: silvia/apps/agnostic-app-test/0.0.19/agnostic-app-test.zip
size: 888
type: file
url: tapis://designsafe.storage.default/silvia/apps/agnostic-app-test/0.0.19/agnostic-app-test.zip


group: 819066
lastModified: 2025-08-19T22:57:07Z
mimeType: application/json
name: app.json
nativePermissions: rw-rw----
owner: 843714
path: silvia/apps/agnostic-app-test/0.0.19/app.json
size: 4250
type: file
url: tapis://designsafe.storage.default/silvia/apps/agnostic-app-test/0.0.19/app.json


group: 819066
lastModified: 2025-08-19T22:57:07Z
mimeType: application/json
name: profile.json
nativePermissions: rw-rw----
owner: 843714
path: silvia/apps/agnostic-app-test/0.0.19/profile.json
size: 178
type: file
url: tapis://designsafe.storage.default/silvia/apps/agnostic-app-t

---
## Register the App
This creates the actual App record that Jobs can run.

Do this Using Tapipy (Python)

In [25]:
if do_makeApp:
    # Create (or create a new version) of the app
    with open(f'{app_folder}/app.json') as f:
        app_def = json.load(f)
    t.apps.createAppVersion(**app_def)

### Check that app is up

#### List all apps

In [26]:
listType = 'ALL' # Include all items requester is authorized to view. Includes check for READ or MODIFY permission.
select = 'id,created,description,version,owner' # Attributes to return in each result.
orderBy = 'created(asc)'
results = t.apps.getApps( orderBy=orderBy,
                         select=select)  
for thisRes in results:
    print('--')
    print(thisRes)

--

created: 2024-12-13T20:15:33.014747Z
description: Run shell commands on remote systems
id: shell-runner-1.0.0
owner: silvia
version: 1.0.0
--

created: 2024-12-14T15:49:21.510206Z
description: Run shell commands on remote systems
id: shell-runner
owner: silvia
version: 1.0.0
--

created: 2025-08-16T16:51:35.355715Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3-silvia
owner: silvia
version: latest
--

created: 2025-08-16T18:53:58.422294Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3-silvia-new
owner: silvia
version: latest
--

created: 2025-08-17T22:34:01.193589Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3-copy-silvia
owner: silvia
vers

#### List the new app

In [27]:
appMetaData = t.apps.getAppLatestVersion(appId=app_id)
print(appMetaData)


containerImage: tapis://designsafe.storage.default/silvia/apps/agnostic-app-test/0.0.19/agnostic-app-test.zip
created: 2025-08-19T22:57:07.742054Z
deleted: False
description: Run any Executable
enabled: True
id: agnostic-app-test
isPublic: False
jobAttributes: 
archiveOnAppError: True
archiveSystemDir: HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}
archiveSystemId: stampede3
cmdPrefix: None
coresPerNode: 48
description: None
dtnSystemInputDir: !tapis_not_set
dtnSystemOutputDir: !tapis_not_set
dynamicExecSystem: False
execSystemConstraints: None
execSystemExecDir: ${JobWorkingDir}
execSystemId: stampede3
execSystemInputDir: ${JobWorkingDir}
execSystemLogicalQueue: skx
execSystemOutputDir: ${JobWorkingDir}
fileInputArrays: []
fileInputs: [
autoMountLocal: True
description: Input directory that includes the input script as well as any other required files. Example input is in tapis://designsafe.storage.community/app_examples/opensees/OpenSeesMP
envKey: inputDi

---
## Submit a Job

You can now submit a job using this app. You can use the Tapis CLI, Tapipy, or a web form.

We are using TapiPy directly from this notebook. We will not specify a version so that the latest is used by default in the description.

---
### Initialize

In [28]:
tapisInputAll = {}

---
### SLURM-Specific Input

In [29]:
tapisInputAll["maxMinutes"] = 6

tapisInputAll["execSystemId"] = "stampede3"
tapisInputAll["execSystemLogicalQueue"] = "skx-dev"
tapisInputAll["nodeCount"] = 1
tapisInputAll["coresPerNode"] = 48
tapisInputAll["allocation"] = "DS-HPC1"

tapisInputAll['archive_system']='Work' # Options: MyData or Work

---
### App-Specific Input

In [30]:
print('app_id:',app_id)

app_id: agnostic-app-test


### OpenSees

In [32]:
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 

tapisInput['main_program'] = 'OpenSees'
tapisInput['input_filename'] = 'Ex1a.Canti2D.Push.tcl'

tapisInput['command_line_args'] = '[33,22]'

# -----------------------------------------------------
if do_submitJob:
    for hereApp in ['opensees-express',app_id]:
        here_tapisInput = tapisInput.copy()
        here_tapisInput["appId"] = hereApp 
        here_tapisInput["name"] = here_tapisInput['main_program'] + '_' + here_tapisInput["appId"]
        print(f'\n -- {here_tapisInput['name']} --\n')
        # jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
        if hereApp == app_id:
            here_tapisInput['zipFolderOut'] = 'True'
            here_tapisInput["name"] = here_tapisInput['main_program'] + '_' + here_tapisInput["appId"] + '_' + 'zipFolderOut'
            print(f'\n -- {here_tapisInput['name']} --\n')
            jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------


 -- OpenSees_opensees-express --


 -- OpenSees_agnostic-app-test --


 -- OpenSees_agnostic-app-test_zipFolderOut --

Creating job_description
Submitting Job
Job submitted! ID: c19962b0-5d8f-4d91-9ca3-ed06c1f926fc-007
job_start_time: 1755645035.1395204

Real-Time Job-Status Updates...
--------------------
	 Elapsed job time: 1.05 sec	 Current Status: PENDING
	 Elapsed job time: 2.08 sec	 Current Status: PROCESSING_INPUTS		(PENDING took 1.03 sec)
	 Elapsed job time: 5.15 sec	 Current Status: STAGING_INPUTS		(PROCESSING_INPUTS took 3.07 sec)
	 Elapsed job time: 41.15 sec	 Current Status: STAGING_JOB		(STAGING_INPUTS took 36.01 sec)
	 Elapsed job time: 73.03 sec	 Current Status: SUBMITTING_JOB		(STAGING_JOB took 31.88 sec)
	 Elapsed job time: 74.06 sec	 Current Status: QUEUED		(SUBMITTING_JOB took 1.03 sec)
	 Elapsed job time: 76.11 sec	 Current Status: RUNNING		(QUEUED took 2.05 sec)
	 Elapsed job time: 101.89 sec	 Current Status: ARCHIVING		(RUNNING took 25.78 sec)
	 Elapsed job time:

Accordion(children=(Output(),), selected_index=0, titles=('Job STATUS   (c19962b0-5d8f-4d91-9ca3-ed06c1f926fc-…

Accordion(children=(Output(),), selected_index=0, titles=('Job Metadata   (c19962b0-5d8f-4d91-9ca3-ed06c1f926f…

Accordion(children=(Output(),), selected_index=0, titles=('Job History Data   (c19962b0-5d8f-4d91-9ca3-ed06c1f…

Accordion(children=(Output(),), selected_index=0, titles=('Job Filedata   (c19962b0-5d8f-4d91-9ca3-ed06c1f926f…

### OpenSeesMP

In [33]:
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 

tapisInput['main_program'] = 'OpenSeesMP'
tapisInput['input_filename'] = 'Ex1a_many.Canti2D.Push.mp.tcl'

tapisInput['command_line_args'] = '[33,22]'

# -----------------------------------------------------
if do_submitJob:
    for hereApp in ['opensees-mp-s3',app_id]:
        here_tapisInput = tapisInput.copy()
        here_tapisInput["appId"] = hereApp 
        here_tapisInput["name"] = here_tapisInput['main_program'] + '_' + here_tapisInput["appId"]
        print(f'\n -- {here_tapisInput['name']} --\n')
        jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
        if hereApp == app_id:
            here_tapisInput['zipFolderOut'] = 'True'
            here_tapisInput["name"] = here_tapisInput['main_program'] + '_' + here_tapisInput["appId"] + '_' + 'zipFolderOut'
            print(f'\n -- {here_tapisInput['name']} --\n')
            jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------


 -- OpenSeesMP_opensees-mp-s3 --

Creating job_description
Submitting Job
Job submitted! ID: dc4c3b70-4070-4b53-b60a-d68b91ef3461-007
job_start_time: 1755645588.867304

Real-Time Job-Status Updates...
--------------------
	 Elapsed job time: 1.05 sec	 Current Status: PENDING
	 Elapsed job time: 2.07 sec	 Current Status: PROCESSING_INPUTS		(PENDING took 1.03 sec)
	 Elapsed job time: 3.1 sec	 Current Status: STAGING_INPUTS		(PROCESSING_INPUTS took 1.03 sec)
	 Elapsed job time: 39.1 sec	 Current Status: STAGING_JOB		(STAGING_INPUTS took 36.01 sec)
	 Elapsed job time: 60.77 sec	 Current Status: SUBMITTING_JOB		(STAGING_JOB took 21.67 sec)
	 Elapsed job time: 61.79 sec	 Current Status: QUEUED		(SUBMITTING_JOB took 1.02 sec)
	 Elapsed job time: 63.85 sec	 Current Status: RUNNING		(QUEUED took 2.06 sec)
	 Elapsed job time: 89.61 sec	 Current Status: ARCHIVING		(RUNNING took 25.76 sec)
	 Elapsed job time: 142.44 sec	 Current Status: FINISHED		(ARCHIVING took 52.83 sec)
	  Status: FINISHED	 El

Accordion(children=(Output(),), selected_index=0, titles=('Job STATUS   (dc4c3b70-4070-4b53-b60a-d68b91ef3461-…

Accordion(children=(Output(),), selected_index=0, titles=('Job Metadata   (dc4c3b70-4070-4b53-b60a-d68b91ef346…

Accordion(children=(Output(),), selected_index=0, titles=('Job History Data   (dc4c3b70-4070-4b53-b60a-d68b91e…

Accordion(children=(Output(),), selected_index=0, titles=('Job Filedata   (dc4c3b70-4070-4b53-b60a-d68b91ef346…

...


 -- OpenSeesMP_agnostic-app-test_zipFolderOut --

Creating job_description
Submitting Job
Job submitted! ID: 2aa1e21b-9289-4410-afaa-768d2397ee9b-007
job_start_time: 1755645884.7424004

Real-Time Job-Status Updates...
--------------------
	 Elapsed job time: 1.04 sec	 Current Status: PENDING
	 Elapsed job time: 2.06 sec	 Current Status: STAGING_INPUTS		(PENDING took 1.02 sec)
	 Elapsed job time: 27.76 sec	 Current Status: STAGING_JOB		(STAGING_INPUTS took 25.7 sec)
	 Elapsed job time: 50.43 sec	 Current Status: QUEUED		(STAGING_JOB took 22.68 sec)
	 Elapsed job time: 52.48 sec	 Current Status: RUNNING		(QUEUED took 2.04 sec)
	 Elapsed job time: 67.87 sec	 Current Status: ARCHIVING		(RUNNING took 15.39 sec)
	 Elapsed job time: 95.77 sec	 Current Status: FINISHED		(ARCHIVING took 27.9 sec)
	  Status: FINISHED	 Elapsed job time: 95.77 sec
--------------------
Elapsed time since Job was submitted: 95.77 sec
--------------------


Accordion(children=(Output(),), selected_index=0, titles=('Job STATUS   (2aa1e21b-9289-4410-afaa-768d2397ee9b-…

Accordion(children=(Output(),), selected_index=0, titles=('Job Metadata   (2aa1e21b-9289-4410-afaa-768d2397ee9…

Accordion(children=(Output(),), selected_index=0, titles=('Job History Data   (2aa1e21b-9289-4410-afaa-768d239…

Accordion(children=(Output(),), selected_index=0, titles=('Job Filedata   (2aa1e21b-9289-4410-afaa-768d2397ee9…

### OpenSeesSP

In [34]:
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 

tapisInput['main_program'] = 'OpenSeesSP'
tapisInput['input_filename'] = 'simpleSP.tcl'

tapisInput['command_line_args'] = '[33,22]'

# -----------------------------------------------------
if do_submitJob:
    for hereApp in ['opensees-sp-s3',app_id]:
        here_tapisInput = tapisInput.copy()
        here_tapisInput["appId"] = hereApp 
        here_tapisInput["name"] = here_tapisInput['main_program'] + '_' + here_tapisInput["appId"]
        print(f'\n -- {here_tapisInput['name']} --\n')
        jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
        if hereApp == app_id:
            here_tapisInput['zipFolderOut'] = 'True'
            here_tapisInput["name"] = here_tapisInput['main_program'] + '_' + here_tapisInput["appId"] + '_' + 'zipFolderOut'
            print(f'\n -- {here_tapisInput['name']} --\n')
            jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------


 -- OpenSeesSP_opensees-sp-s3 --

Creating job_description
Submitting Job
Job submitted! ID: 2886e3d5-87cd-48d2-b61b-b39e435fe850-007
job_start_time: 1755648124.668872

Real-Time Job-Status Updates...
--------------------
	 Elapsed job time: 1.06 sec	 Current Status: PENDING
	 Elapsed job time: 2.08 sec	 Current Status: PROCESSING_INPUTS		(PENDING took 1.02 sec)
	 Elapsed job time: 5.21 sec	 Current Status: STAGING_INPUTS		(PROCESSING_INPUTS took 3.12 sec)
	 Elapsed job time: 41.15 sec	 Current Status: STAGING_JOB		(STAGING_INPUTS took 35.94 sec)
	 Elapsed job time: 73.01 sec	 Current Status: QUEUED		(STAGING_JOB took 31.86 sec)
	 Elapsed job time: 86.41 sec	 Current Status: RUNNING		(QUEUED took 13.41 sec)
	 Elapsed job time: 98.85 sec	 Current Status: ARCHIVING		(RUNNING took 12.44 sec)
	 Elapsed job time: 142.32 sec	 Current Status: FINISHED		(ARCHIVING took 43.47 sec)
	  Status: FINISHED	 Elapsed job time: 142.32 sec
--------------------
Elapsed time since Job was submitted: 142.3

Accordion(children=(Output(),), selected_index=0, titles=('Job STATUS   (2886e3d5-87cd-48d2-b61b-b39e435fe850-…

Accordion(children=(Output(),), selected_index=0, titles=('Job Metadata   (2886e3d5-87cd-48d2-b61b-b39e435fe85…

Accordion(children=(Output(),), selected_index=0, titles=('Job History Data   (2886e3d5-87cd-48d2-b61b-b39e435…

Accordion(children=(Output(),), selected_index=0, titles=('Job Filedata   (2886e3d5-87cd-48d2-b61b-b39e435fe85…

...


 -- OpenSeesSP_agnostic-app-test_zipFolderOut --

Creating job_description
Submitting Job
Job submitted! ID: 0f3185a3-d71f-411e-a93a-2e8e261a7557-007
job_start_time: 1755648400.4160562

Real-Time Job-Status Updates...
--------------------
	 Elapsed job time: 1.04 sec	 Current Status: PENDING
	 Elapsed job time: 2.07 sec	 Current Status: STAGING_INPUTS		(PENDING took 1.03 sec)
	 Elapsed job time: 38.08 sec	 Current Status: STAGING_JOB		(STAGING_INPUTS took 36.01 sec)
	 Elapsed job time: 69.88 sec	 Current Status: SUBMITTING_JOB		(STAGING_JOB took 31.8 sec)
	 Elapsed job time: 70.9 sec	 Current Status: QUEUED		(SUBMITTING_JOB took 1.02 sec)
	 Elapsed job time: 72.95 sec	 Current Status: RUNNING		(QUEUED took 2.05 sec)
	 Elapsed job time: 98.79 sec	 Current Status: ARCHIVING		(RUNNING took 25.84 sec)
	 Elapsed job time: 142.23 sec	 Current Status: FINISHED		(ARCHIVING took 43.44 sec)
	  Status: FINISHED	 Elapsed job time: 142.23 sec
--------------------
Elapsed time since Job was submitt

Accordion(children=(Output(),), selected_index=0, titles=('Job STATUS   (0f3185a3-d71f-411e-a93a-2e8e261a7557-…

Accordion(children=(Output(),), selected_index=0, titles=('Job Metadata   (0f3185a3-d71f-411e-a93a-2e8e261a755…

Accordion(children=(Output(),), selected_index=0, titles=('Job History Data   (0f3185a3-d71f-411e-a93a-2e8e261…

Accordion(children=(Output(),), selected_index=0, titles=('Job Filedata   (0f3185a3-d71f-411e-a93a-2e8e261a755…

### OpenSeesPy

In [ ]:
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 

tapisInput['main_program'] = 'python'
tapisInput['input_filename'] = 'Ex1a.Canti2D.Push.mpi.py'

tapisInput['command_line_args'] = '[33,22]'

# -----------------------------------------------------
if do_submitJob:
    for hereApp in [app_id]:
        here_tapisInput = tapisInput.copy()
        here_tapisInput["appId"] = hereApp 
        here_tapisInput["name"] = here_tapisInput['main_program'] + '_' + here_tapisInput["appId"]
        print(f'\n -- {here_tapisInput['name']} --\n')
        jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
        if hereApp == app_id:
            here_tapisInput['zipFolderOut'] = 'True'
            here_tapisInput["name"] = here_tapisInput['main_program'] + '_' + here_tapisInput["appId"] + '_' + 'zipFolderOut'
            print(f'\n -- {here_tapisInput['name']} --\n')
            jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------


 -- python_agnostic-app-test --

Creating job_description
Submitting Job
Job submitted! ID: 0ff17cb2-b595-4c99-b76a-f7e46396477e-007
job_start_time: 1755648550.427651

Real-Time Job-Status Updates...
--------------------
	 Elapsed job time: 1.07 sec	 Current Status: PENDING
	 Elapsed job time: 2.12 sec	 Current Status: STAGING_INPUTS		(PENDING took 1.04 sec)
	 Elapsed job time: 27.88 sec	 Current Status: STAGING_JOB		(STAGING_INPUTS took 25.76 sec)
	 Elapsed job time: 59.78 sec	 Current Status: SUBMITTING_JOB		(STAGING_JOB took 31.9 sec)
	 Elapsed job time: 60.8 sec	 Current Status: QUEUED		(SUBMITTING_JOB took 1.02 sec)
	 Elapsed job time: 74.15 sec	 Current Status: RUNNING		(QUEUED took 13.35 sec)
	 Elapsed job time: 152.42 sec	 Current Status: ARCHIVING		(RUNNING took 78.26 sec)
	 Elapsed job time: 198.01 sec	 Current Status: FINISHED		(ARCHIVING took 45.59 sec)
	  Status: FINISHED	 Elapsed job time: 198.01 sec
--------------------
Elapsed time since Job was submitted: 198.01 sec
-

Accordion(children=(Output(),), selected_index=0, titles=('Job STATUS   (0ff17cb2-b595-4c99-b76a-f7e46396477e-…

Accordion(children=(Output(),), selected_index=0, titles=('Job Metadata   (0ff17cb2-b595-4c99-b76a-f7e46396477…

Accordion(children=(Output(),), selected_index=0, titles=('Job History Data   (0ff17cb2-b595-4c99-b76a-f7e4639…

Accordion(children=(Output(),), selected_index=0, titles=('Job Filedata   (0ff17cb2-b595-4c99-b76a-f7e46396477…


 -- python_agnostic-app-test_zipFolderOut --

Creating job_description
Submitting Job
Job submitted! ID: a0dc3b5e-72f8-4cee-ba21-1ed232865718-007
job_start_time: 1755648757.083099

Real-Time Job-Status Updates...
--------------------
	 Elapsed job time: 1.04 sec	 Current Status: PENDING
	 Elapsed job time: 2.08 sec	 Current Status: STAGING_INPUTS		(PENDING took 1.04 sec)


### OpenSeesPy with zipFolderOut

In [ ]:
# tapisInput = tapisInputAll.copy()

# tapisInput['zipFolderOut'] = 'True'

# tapisInput['storage_system'] = 'CommunityData'
# tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 

# tapisInput['main_program'] = 'python'
# tapisInput['input_filename'] = 'Ex1a.Canti2D.Push.mpi.py'

# tapisInput['command_line_args'] = '[33,22]'

# # -----------------------------------------------------
# if do_submitJob:
#     for hereApp in [app_id]:
#         here_tapisInput = tapisInput.copy()
#         here_tapisInput["appId"] = hereApp 
#         here_tapisInput["name"] = here_tapisInput['main_program'] + '_' + here_tapisInput["appId"]
#         print(f'\n -- {here_tapisInput['name']} --\n')
#         jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
#         if hereApp == app_id:
#             here_tapisInput['zipFolderOut'] = 1
#             here_tapisInput["name"] = here_tapisInput['main_program'] + '_' + here_tapisInput["appId"] + '_' + 'zipFolderOut'
#             print(f'\n -- {here_tapisInput['name']} --\n')
#             jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# # -----------------------------------------------------